In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv('../../../data/kc_house_data_with_dummies.csv')

In [3]:
def relevants(data):
    rel = []
    for col in data.columns:
        if data[col].dtype != object:
            rel.append(col)
    return rel

def graphs(data):
    relevant = relevants(data.drop('price', axis = 1))
    rowsplot = int(len(relevant)/2 + 1)

    fig, axes = plt.subplots(nrows = rowsplot, ncols = 2, figsize=(20, 130))

    for col, ax in zip(relevant, axes.flatten()):
        sns.scatterplot(data = data, x = col, y = 'price', ax = ax)
        ax.set_title(col, fontsize = 20)
        
def make_corrs(data, display = False):
    corrs = data.corr().stack().reset_index()
    corrs = corrs.loc[corrs['level_0'] != corrs['level_1']]
    corrs['pair'] = corrs['level_0'] + ' '+ corrs['level_1']
    corrs['pair'] = corrs['pair'].map(lambda x: ', '.join(sorted(x.split(' '))))
    corrs.drop_duplicates(subset = 'pair', inplace = True)
    corrs.drop(['level_1','level_0'], axis = 1, inplace = True)
    corrs.set_index('pair', inplace = True)
    corrs.columns = pd.Series('correlation')
    corrs = abs(corrs).sort_values('correlation', ascending = False)
    if display:
        display(corrs)
    return corrs

def make_high_corrs(data, display = False):
    corrs = make_corrs(data)
    high_corrs = corrs.loc[corrs['correlation']>= 0.75]
    if display:
        display(high_corrs)
    return high_corrs

def make_corrs_with_price(data, display = False):
    corrs = make_corrs(data)
    corrs_with_price = corrs.loc[corrs.index.str.contains('price')]
    if display:
        display(corrs_with_price)
    return corrs_with_price

def check_homoscedastic(data, display = False):
    results = []
    for col in data.drop('price', axis = 1).columns:
        lower = data[col].quantile(0.45)
        upper = data[col].quantile(0.55)
        mids = data.loc[(data[col] >= lower)&(data[col] <= upper)].index

        index = [x for x in data.index if x not in mids]
        formula = f'price~{col}'
        model = smf.ols(formula = formula, data = data).fit()

        results.append((col, sms.het_goldfeldquandt(model.resid.iloc[index], model.model.exog[index])[1]))

    heteroscedastic = [(col, p) for col, p in results if p < 0.05]
    if display:
        print(heteroscedastic)
    
    return heteroscedastic, results

def corr(x, y):
    xbar = x.mean()
    ybar = y.mean()
    return sum((x-xbar)*(y-ybar))/np.sqrt(sum((x-xbar)**2)*sum((y-ybar)**2))



In [4]:
poly = PolynomialFeatures(2)
data_cop = data.drop(['price','date','zipcode','datefloat','act_prce','id'], axis = 1)
poly.fit(data_cop)
X = pd.DataFrame(poly.transform(data_cop), columns = poly.get_feature_names(data_cop.columns), index = data_cop.index)
y = data['price']
print(len(X.columns))
linreg = LinearRegression()

4560


In [5]:
print(X.columns)

Index(['1', 'sqft_lot', 'grade', 'sqft_above', 'sqft_basement', 'yr_built',
       'yr_renovated', 'lat', 'long', 'sqft_living15',
       ...
       'condition_4^2', 'condition_4 condition_5', 'condition_4 renovated_0',
       'condition_4 renovated_1', 'condition_5^2', 'condition_5 renovated_0',
       'condition_5 renovated_1', 'renovated_0^2', 'renovated_0 renovated_1',
       'renovated_1^2'],
      dtype='object', length=4560)


In [6]:
categoricals = ['bedrooms','bathrooms','floors','waterfront','view','condition','renovated']

In [7]:
dropped = 0
for cat in categoricals:
    cols = X.columns[X.columns.str.contains(cat)]
    for col in cols:
        if len(col.split(' ')) == 2:
            factors = col.split(' ')
            if factors[0].split('_')[0] == factors[1].split('_')[0]:
                print(f'Removing {col}', end = '--')
                dropped += 1
                X.drop(col, axis = 1, inplace = True)
                
        else:
            if '^' in col:
                print(f'Removing {col}', end = '--')
                dropped += 1
                X.drop(col, axis = 1, inplace = True)

Removing bedrooms_1^2--Removing bedrooms_1 bedrooms_2--Removing bedrooms_1 bedrooms_3--Removing bedrooms_1 bedrooms_4--Removing bedrooms_1 bedrooms_5--Removing bedrooms_1 bedrooms_6--Removing bedrooms_1 bedrooms_7--Removing bedrooms_1 bedrooms_8--Removing bedrooms_1 bedrooms_9--Removing bedrooms_1 bedrooms_10--Removing bedrooms_1 bedrooms_11--Removing bedrooms_2^2--Removing bedrooms_2 bedrooms_3--Removing bedrooms_2 bedrooms_4--Removing bedrooms_2 bedrooms_5--Removing bedrooms_2 bedrooms_6--Removing bedrooms_2 bedrooms_7--Removing bedrooms_2 bedrooms_8--Removing bedrooms_2 bedrooms_9--Removing bedrooms_2 bedrooms_10--Removing bedrooms_2 bedrooms_11--Removing bedrooms_3^2--Removing bedrooms_3 bedrooms_4--Removing bedrooms_3 bedrooms_5--Removing bedrooms_3 bedrooms_6--Removing bedrooms_3 bedrooms_7--Removing bedrooms_3 bedrooms_8--Removing bedrooms_3 bedrooms_9--Removing bedrooms_3 bedrooms_10--Removing bedrooms_3 bedrooms_11--Removing bedrooms_4^2--Removing bedrooms_4 bedrooms_5--Removi

In [8]:
dropped

560

In [9]:
X.columns

Index(['1', 'sqft_lot', 'grade', 'sqft_above', 'sqft_basement', 'yr_built',
       'yr_renovated', 'lat', 'long', 'sqft_living15',
       ...
       'condition_1 renovated_0', 'condition_1 renovated_1',
       'condition_2 renovated_0', 'condition_2 renovated_1',
       'condition_3 renovated_0', 'condition_3 renovated_1',
       'condition_4 renovated_0', 'condition_4 renovated_1',
       'condition_5 renovated_0', 'condition_5 renovated_1'],
      dtype='object', length=4000)

In [10]:
dropped = 0
for col in X.columns:
    if X[col].nunique() == 1:
        X.drop(col, axis = 1, inplace = True)
        dropped+=1

In [11]:
dropped

646

In [12]:
X.columns

Index(['sqft_lot', 'grade', 'sqft_above', 'sqft_basement', 'yr_built',
       'yr_renovated', 'lat', 'long', 'sqft_living15', 'sqft_lot15',
       ...
       'condition_1 renovated_0', 'condition_1 renovated_1',
       'condition_2 renovated_0', 'condition_2 renovated_1',
       'condition_3 renovated_0', 'condition_3 renovated_1',
       'condition_4 renovated_0', 'condition_4 renovated_1',
       'condition_5 renovated_0', 'condition_5 renovated_1'],
      dtype='object', length=3354)

In [13]:
def five_fold(X, y, say = False):
    if say:
        print('KFOLDS')
    folds = KFold(n_splits = 5, shuffle = True)
    trains = []
    tests = []
    count = 0
    for tr_ind, te_ind in folds.split(X):
        X_train, X_test = X.iloc[tr_ind], X.iloc[te_ind]
        y_train, y_test = y.iloc[tr_ind], y.iloc[te_ind]
        linreg.fit(X_train, y_train)
        trains.append(linreg.score(X_train, y_train))
        tests.append(linreg.score(X_test, y_test))
        count +=1

    return sum(trains)/5, sum(tests)/5

In [14]:
import copy
def find_best(X, y, initial):
    while 1:
        changed = 0
        init_train, init_test = five_fold(X[initial], y)
        print(initial)
        addition = 0
        added = ''
        for col in X.drop(initial, axis = 1).columns:
            new = copy.copy(initial)
            new.append(col)
            train_sc, test_sc = five_fold(X[new], y)
            if (test_sc > addition) and (test_sc > init_test) and (abs(test_sc - train_sc) < 0.07):
                addition = test_sc
                added = col
        if addition != 0:
            changed = 1
            initial.append(added)
            print(f'Added {added} with score {addition}')
        
        init_train, init_test = five_fold(X[initial], y)
        print(initial)
        removal = 0
        removed = ''
        for col in initial:
            new = copy.copy(initial)
            new.remove(col)
            train_sc, test_sc = five_fold(X[new], y)
            if (test_sc > removal) and (test_sc > init_test) and (abs(test_sc - train_sc) < 0.07):
                removal = test_sc
                removed = col
        if removal != 0:
            changed = 1
            initial.remove(removed)
            print(f'Removed {removed} with score {removal}')
        
        if changed == 0:
            break
    print(initial)

In [15]:
old_inits = ['grade', 'log_sqft_living', 'median_prce comfort_index', 'median_prce^2', 'years_since_new^2', 'yr_built', 'sqft_lot yr_built', 'waterfront log_sqft_living', 'median_income^2', 'grade median_income', 'floors bath_per_bed', 'sqft_lot median_prce', 'sqft_lot^2', 'median_income median_prce', 'median_income comfort_index', 'comfort_index^2', 'yr_built comfort_index', 'median_age median_prce', 'grade lat', 'skinniness years_since_new', 'condition median_age', 'yr_renovated log_sqft_living', 'sqft_living15 years_since_new', 'grade log_sqft_living', 'bathrooms sqft_lot', 'condition median_income', 'sqft_above bath_per_bed', 'years_since_new log_sqft_living', 'long month_1', 'grade month_2', 'view long', 'lat median_income', 'view dist', 'view median_prce', 'unemployment years_since_new', 'view elevation', 'lat', 'median_prce years_since_new', 'condition sqft_living15', 'dist median_prce', 'dist comfort_index', 'waterfront elevation', 'grade long', 'floors month_6', 'unemployment median_income', 'floors years_since_new', 'bath_per_bed median_income', 'sqft_living15 month_3', 'sqft_above median_prce', 'yr_built renovated', 'sqft_lot yr_renovated', 'sqft_lot month_5']

In [16]:
inits = ['grade', 'log_sqft_living', 'median_prce comfort_index', 'median_prce^2', 'years_since_new^2', 'unemployment years_since_new', 'lat^2', 'sqft_lot yr_renovated', 'sqft_above bath_per_bed', 'comfort_index month_4', 'sqft_living15', 'unemployment view_4.0', 'sqft_lot median_prce', 'lat', 'condition_5 renovated_0', 'yr_renovated condition_2', 'grade long', 'sqft_lot^2', 'yr_built comfort_index', 'bath_per_bed^2', 'grade condition_1', 'median_income view_0.0', 'lat waterfront_0', 'grade comfort_index', 'skinniness years_since_new', 'log_sqft_living view_3.0', 'median_income^2', 'bath_per_bed floors_1.0', 'sqft_living15 years_since_new', 'median_income median_prce', 'median_age median_prce', 'lat view_0.0', 'median_age waterfront_0', 'median_prce floors_3.0', 'dist view_0.0', 'sqft_lot15 bedrooms_5', 'yr_renovated sqft_living15', 'median_age month_3', 'comfort_index bedrooms_4', 'sqft_living15 bedrooms_9', 'bedrooms_1 floors_1.0', 'lat condition_3', 'month_11 bedrooms_8', 'median_prce bathrooms_6.25', 'skinniness elevation']
find_best(X, y, inits)

['grade', 'log_sqft_living', 'median_prce comfort_index', 'median_prce^2', 'yr_built waterfront_0', 'years_since_new^2', 'unemployment years_since_new', 'lat^2', 'sqft_lot yr_renovated', 'condition_3 renovated_0', 'sqft_above bath_per_bed', 'comfort_index month_4', 'sqft_living15', 'unemployment view_4.0', 'sqft_lot median_prce', 'lat', 'yr_built sqft_living15', 'condition_5 renovated_0', 'yr_renovated condition_2', 'median_age month_3', 'grade long', 'median_age view_0.0', 'sqft_lot^2', 'sqft_living15 median_income', 'yr_built comfort_index']
Added bath_per_bed^2 with score 0.8736355550185493
['grade', 'log_sqft_living', 'median_prce comfort_index', 'median_prce^2', 'yr_built waterfront_0', 'years_since_new^2', 'unemployment years_since_new', 'lat^2', 'sqft_lot yr_renovated', 'condition_3 renovated_0', 'sqft_above bath_per_bed', 'comfort_index month_4', 'sqft_living15', 'unemployment view_4.0', 'sqft_lot median_prce', 'lat', 'yr_built sqft_living15', 'condition_5 renovated_0', 'yr_ren

KeyboardInterrupt: 